In [ ]:
def reproduceResult():
  seed_value= 0

  
  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

  


In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import os 
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow import keras

reproduceResult()
import tempfile
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K


from tensorflow.keras import callbacks

from keras_lr_finder import LRFinder
from clr_callback import CyclicLR

import tensorflow_model_optimization as tfmot



import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
def clean_punc(line):
  temp = re.sub(r'[?|!|\'|"|#]',r'',line)
  temp = re.sub(r'[.|,|"|")|(|\|/]',r' ',temp)
  return  temp

def clean_html(line):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, ' ', line)
  return cleantext

def clean_space(line):
  return " ".join(line.split())


def clean_char(line):
  clean_ch = re.compile('[!@#$%^&*()_+-={}\[\];:\'\"\|<>,.///?`~।]', flags=re.I)
  return clean_ch.sub(r'',line)


def clean_non_bn_words(line):
  clean_en = re.compile('[a-z]', flags=re.I)
  clean_en_up = re.compile('[A-Z]', flags=re.I)
  clean_digit = re.compile('[0-9]', flags=re.I)
  line = clean_en.sub(r'',line)
  line = clean_en_up.sub(r'',line)
  return clean_digit.sub(r'',line)

def clean_emo(line):

  result = re.sub('[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)]+','',line)
  return result


def clean_all(line):
  line = clean_html(line)
  line = clean_punc(line)
  line = clean_char(line)
  line = clean_non_bn_words(line)
  line = clean_space(line)
  return clean_emo(line)


def float_to_int(val):
  return int(val)



In [ ]:
import re

df1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/DeepHateExplainer/train.csv', sep=',', encoding='utf-8')
df2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/DeepHateExplainer/test.csv', sep=',', encoding='utf-8')
df3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/DeepHateExplainer/validation.csv', sep=',', encoding='utf-8')


df1 = df1.dropna()

df1['Text'] = df1['text'].apply(clean_all)
df2['Text'] = df2['text'].apply(clean_all)
df3['Text'] = df3['text'].apply(clean_all)




del df1['text']
del df1['target']
del df2['text']
del df2['target']
del df3['text']
del df3['target']


frames = [df1, df2, df3]

temp = pd.concat(frames)

print(temp)

temp = temp.sample(frac=1).reset_index(drop=True)

print(temp)


            label                                               Text
0        Personal          ৭৫ হামলার আসামী কে এই গডফাদার কালা মন্দির
1       Political  রাজাকারদের মন্ত্রী বানানোর দায়ে তার আরও কঠিন ...
2    Geopolitical  বৃটিশ নাগরিকদের বাংলাদেশ ভ্রমনে সতর্কতা জারী ২...
3       Political  বলদ নয়াদিগন্ত কালকে রিপোর্ট করলো হিলারী শপথ ন...
4    Geopolitical  অসভ্য বর্বর বেহায়া ভোট ডাকাত সকল প্রকার খারাপ...
..            ...                                                ...
995  Geopolitical            আমরা লুটে পুটে খাই তাই আনন্দের সীমা নাই
996      Personal  সব মিমারদের চরিত্র খারাপ তাতে আমার বাল ছিড়া গেছে
997      Personal  এই ম্যাইয়ারে হেমায়েতপুর থেইকা খোলা অবস্থায় ...
998      Personal  অভিজিৎ রায় হত্যাকান্ড খুনিরা দিব্যি চলে গেল প...
999  Geopolitical      পুরো আরবরা কেন পরাজিত আরবদের ইতিহাস জানতে হবে

[8087 rows x 2 columns]
             label                                               Text
0     Geopolitical  রোহিঙ্গাদের ওপর মরিচের গুড়া ও গ্রেনেড ছুড়ছে ...
1      

In [ ]:
temp.label.value_counts()

Personal        3513
Geopolitical    2364
Religious       1211
Political        999
Name: label, dtype: int64

In [ ]:
train, test = train_test_split(temp, test_size=0.2, stratify = temp['label'], random_state = 42)


In [ ]:
train.label.value_counts()

Personal        2810
Geopolitical    1891
Religious        969
Political        799
Name: label, dtype: int64

In [ ]:
num_classes = 4
embed_num_dims = 300
max_seq_len = 50



x_train = train['Text']
x_test = test['Text']

y_train = train['label']
y_test = test['label']

texts_train = x_train
texts_test = x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['Text'])
tokenizer.fit_on_texts(test['Text'])


sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

index_of_words = tokenizer.word_index

vocab_size = len(index_of_words) + 1

print('Number of unique words: {}'.format(len(index_of_words)))

X_train_pad = pad_sequences(sequence_train, maxlen = max_seq_len, padding='post' )
X_test_pad = pad_sequences(sequence_test, maxlen = max_seq_len,  padding='post')

print(X_train_pad)


encoding = {
    'Personal': 0,
    'Geopolitical': 1,
    'Religious': 2,
    'Political': 3,
    }

y_train = [encoding[x] for x in train['label']]
y_test = [encoding[x] for x in test['label']]



y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('/content/drive/MyDrive/Colab Notebooks/Dataset/cc.bn.300.vec') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)
print(embedd_matrix.shape)




Number of unique words: 19282
[[9068 9069  302 ...    0    0    0]
 [ 137 1248 2878 ...    0    0    0]
 [ 311 1936 1249 ...    0    0    0]
 ...
 [   3 5577  108 ...    0    0    0]
 [ 196   27   46 ...    0    0    0]
 [8857 2155 5391 ...    0    0    0]]
(19283, 300)


In [ ]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0



def build_model(hp):
  
  reproduceResult()

  print('Ya it comes here')
  unit_attention = hp.Int("attention_unit",min_value =32, max_value = 128, step = 16)
  fake_val = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =64, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_unit = hp.Int("cnn_2_unit",min_value =64, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn_2_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)



  seq_input = keras.layers.Input(shape=(max_seq_len,))

  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = max_seq_len,
                          weights = [embedd_matrix])(seq_input)

  cnn = keras.layers.Conv1D(cnn_1_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)

  attention_vec = keras.layers.TimeDistributed(keras.layers.Dense(unit_attention))(cnn)
  attention_vec = keras.layers.Reshape((48,unit_attention))(attention_vec)
  attention_vec = keras.layers.Activation('relu', name = 'cnn_attention_vec')(attention_vec)
  attention_output = keras.layers.Dot(axes = 1)([cnn, attention_vec])


  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_unit, recurrent_regularizer=regularizers.l2(1e-4),
                                                      return_sequences=True,kernel_regularizer=regularizers.l2(1e-4),
                                                      bias_regularizer=regularizers.l2(1e-2),
                                                      activity_regularizer=regularizers.l2(1e-4),input_shape =(48,)))(attention_output)

  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)
  
  

  cnn_2 = keras.layers.Conv1D(cnn_2_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(num_classes, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  return model


stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')


# clr_step_size = int((len(X_train_pad)/16))
# base_lr = 2e-5
# max_lr = 1e-3
# mode = 'exp_range'


# clr = CyclicLR(base_lr = max_lr, max_lr = base_lr, step_size = clr_step_size, mode = mode)



tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 50,
    executions_per_trial = 1,
    directory = LOG_DIR
    )
  
tuner.search(x=X_train_pad,y = y_train,epochs = 20, batch_size = 50,callbacks = [stop], 
             validation_data = (X_test_pad,y_test))


tuner.results_summary()

Trial 28 Complete [00h 06m 26s]
val_accuracy: 0.8114956617355347

Best val_accuracy So Far: 0.8355994820594788
Total elapsed time: 05h 01m 31s

Search: Running Trial #29

Hyperparameter    |Value             |Best Value So Far 
attention_unit    |48                |48                
cnn_1_unit        |32                |48                
cnn_1_dropout     |0.2               |0.2               
lstm_unit         |96                |160               
lstm_dropout      |0.3               |0.1               
cnn_2_unit        |224               |224               
cnn_2_dropout     |0.1               |0.5               

Ya it comes here
Epoch 1/20
120/130 [==========================>...] - ETA: 1s - loss: 3.1721 - accuracy: 0.4070